# Procesamiento de lenguaje natural
## Desafío 2


### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Directorio de los dataset de libros
os.listdir("./libros_dataset/")

['Fundacion.txt']

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('libros_dataset/Fundacion.txt', sep='/n', header=None)
df.head()

/var/folders/tx/kndmxn0s0llcl9y65wfc6y_80000gn/T/ipykernel_8710/1480457263.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('libros_dataset/Fundacion.txt', sep='/n', header=None)


,0
0,TRILOGÍA DE LA FUNDACION
1,FUNDACIÓN
2,FUNDACIÓN E IMPERIO
3,SEGUNDA FUNDACIÓN
4,Reunión en un solo volumen de las tres novelas...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 26966


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['trilogía', 'de', 'la', 'fundacion'], ['fundación']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=12,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 26966


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4351


### 3 - Entrenar embeddings

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 167605.859375
Loss after epoch 1: 132496.140625
Loss after epoch 2: 131799.6875
Loss after epoch 3: 130045.0625
Loss after epoch 4: 123827.6875
Loss after epoch 5: 122599.8125
Loss after epoch 6: 119896.125
Loss after epoch 7: 117862.25
Loss after epoch 8: 107620.375
Loss after epoch 9: 105469.875
Loss after epoch 10: 104578.625
Loss after epoch 11: 103092.5
Loss after epoch 12: 99024.5
Loss after epoch 13: 102581.25
Loss after epoch 14: 99652.5
Loss after epoch 15: 99145.0
Loss after epoch 16: 96760.25
Loss after epoch 17: 96931.375
Loss after epoch 18: 92959.125
Loss after epoch 19: 87917.0
Loss after epoch 20: 86486.75
Loss after epoch 21: 86028.5
Loss after epoch 22: 86384.5
Loss after epoch 23: 86496.5
Loss after epoch 24: 84356.5
Loss after epoch 25: 82456.0
Loss after epoch 26: 85448.75
Loss after epoch 27: 81696.75
Loss after epoch 28: 82041.25
Loss after epoch 29: 82855.25
Loss after epoch 30: 82735.75
Loss after epoch 31: 81755.0
Loss after epoch 32: 81540

(6657559, 10737500)

### 4 - Ensayar

Ensayare el modelo usando las siguientes palabras:

- Imperio.
- Seldon.
- Gaal
- Fundación.
- Psicohistoria.
- Enciclopedia.
- Mulo.
- Términus.
- Imperial.
- Trántor

### Similitudes

Mostraré las 5 palabras mas similes.

In [13]:
# Palabras que MÁS se relacionan con "Imperio":
w2v_model.wv.most_similar(positive=["imperio"], topn=5)


[('humanidad', 0.4108303189277649),
 ('excesivamente', 0.41061973571777344),
 ('galáctico', 0.39687320590019226),
 ('suelta', 0.39265039563179016),
 ('destinada', 0.38818666338920593)]

In [14]:
# Palabras que MÁS se relacionan con "Seldon":
w2v_model.wv.most_similar(positive=["seldon"], topn=5)

[('hari', 0.5696995258331299),
 ('8', 0.4933924973011017),
 ('dornick', 0.4626477360725403),
 ('inminente', 0.42940810322761536),
 ('previó', 0.4185936152935028)]

In [15]:
# Palabras que MÁS se relacionan con "Gaal":
w2v_model.wv.most_similar(positive=["gaal"], topn=5)

[('dornick', 0.4138815701007843),
 ('8', 0.37677448987960815),
 ('aguardó', 0.3741309344768524),
 ('escrita', 0.37136903405189514),
 ('—riose', 0.3686164319515228)]

In [16]:
# Palabras que MÁS se relacionan con "Fundación":
w2v_model.wv.most_similar(positive=["fundación"], topn=5)

[('segunda', 0.47839033603668213),
 ('isaac', 0.4157848358154297),
 ('actualmente', 0.3846505880355835),
 ('convencido', 0.37542980909347534),
 ('protección', 0.3586493134498596)]

In [17]:
# Palabras que MÁS se relacionan con "Psicohistoria":
w2v_model.wv.most_similar(positive=["psicohistoria"], topn=5)

[('distintas', 0.4634544849395752),
 ('reacciones', 0.45407941937446594),
 ('predice', 0.44689813256263733),
 ('sociales', 0.43434205651283264),
 ('leyes', 0.4310802221298218)]

In [18]:
# Palabras que MÁS se relacionan con "Enciclopedia":
w2v_model.wv.most_similar(positive=["enciclopedia"], topn=5)

[('proceden', 0.48386141657829285),
 ('espiral', 0.46682462096214294),
 ('edición', 0.4345708191394806),
 ('f', 0.41842105984687805),
 ('7', 0.41689515113830566)]

In [19]:
# Palabras que MÁS se relacionan con "Mulo":
w2v_model.wv.most_similar(positive=["mulo"], topn=5)

[('tristes', 0.35085055232048035),
 ('mutación', 0.35059884190559387),
 ('maligno', 0.3385881781578064),
 ('—todavía', 0.3377206027507782),
 ('potencial', 0.33400189876556396)]

In [20]:
# Palabras que MÁS se relacionan con "Términus":
w2v_model.wv.most_similar(positive=["términus"], topn=5)

[('establecida', 0.37248238921165466),
 ('llegará', 0.3598547875881195),
 ('estableció', 0.3592778742313385),
 ('estados', 0.35245397686958313),
 ('docena', 0.3459130823612213)]

In [21]:
# Palabras que MÁS se relacionan con "Imperial":
w2v_model.wv.most_similar(positive=["imperial"], topn=5)

[('terrenos', 0.46898820996284485),
 ('etiqueta', 0.41755568981170654),
 ('biblioteca', 0.4109555780887604),
 ('majestad', 0.40965744853019714),
 ('cumplir', 0.39516037702560425)]

In [22]:
# Palabras que MÁS se relacionan con "Trántor":
w2v_model.wv.most_similar(positive=["trántor"], topn=5)

[('santanni', 0.4189485013484955),
 ('setenta', 0.35812363028526306),
 ('humanidad', 0.3569578230381012),
 ('8', 0.33783596754074097),
 ('agrícola', 0.33734655380249023)]

### Diferencias

Mostraré las 5 palabras mas diferentes.

In [23]:
# Palabras que MENOS se relacionan con "Imperio":
w2v_model.wv.most_similar(negative=["imperio"], topn=5)

[('observó', 0.04553065821528435),
 ('parece', 0.04148180037736893),
 ('hacerle', 0.030859841033816338),
 ('incalificable', 0.028506506234407425),
 ('acerca', 0.027717841789126396)]

In [24]:
# Palabras que MENOS se relacionan con "Seldon":
w2v_model.wv.most_similar(negative=["seldon"], topn=5)

[('tus', 0.047680530697107315),
 ('dando', 0.040176574140787125),
 ('vestido', 0.02136993780732155),
 ('atómicos', 0.0211309976875782),
 ('parecían', 0.015290911309421062)]

In [25]:
# Palabras que MENOS se relacionan con "Gaal":
w2v_model.wv.most_similar(negative=["gaal"], topn=5)

[('además', 0.042299263179302216),
 ('cual', 0.030057383701205254),
 ('buena', 0.021691760048270226),
 ('casa', 0.021516995504498482),
 ('uno', 0.019355976954102516)]

In [26]:
# Palabras que MENOS se relacionan con "Fundación":
w2v_model.wv.most_similar(negative=["fundación"], topn=5)

[('receptor', 0.05285129323601723),
 ('delgado', 0.02012752927839756),
 ('frío', 0.019343731924891472),
 ('ven', 0.014842206612229347),
 ('interrumpió', 0.00515055051073432)]

In [27]:
# Palabras que MENOS se relacionan con "Psicohistoria":
w2v_model.wv.most_similar(negative=["psicohistoria"], topn=5)

[('resto', 0.05289734527468681),
 ('muchacho', 0.04732118919491768),
 ('jardín', 0.012435169890522957),
 ('fugitivo', 0.004763633478432894),
 ('recibió', 0.004045072942972183)]

In [28]:
# Palabras que MENOS se relacionan con "Enciclopedia":
w2v_model.wv.most_similar(negative=["enciclopedia"], topn=5)

[('salto', 0.07689458876848221),
 ('repentina', 0.058484215289354324),
 ('cierta', 0.053846560418605804),
 ('contestó', 0.03409111499786377),
 ('atrás', 0.027404405176639557)]

In [29]:
# Palabras que MENOS se relacionan con "Mulo":
w2v_model.wv.most_similar(negative=["mulo"], topn=5)

[('lista', 0.040851544588804245),
 ('placer', 0.03344811871647835),
 ('compañía', 0.029159696772694588),
 ('recibir', 0.02559681050479412),
 ('mañana', 0.023112447932362556)]

In [30]:
# Palabras que MENOS se relacionan con "Términus":
w2v_model.wv.most_similar(negative=["términus"], topn=5)

[('exclamó', 0.041755370795726776),
 ('pesar', 0.041379936039447784),
 ('falta', 0.029171321541070938),
 ('barba', 0.029141239821910858),
 ('emociones', 0.02095438539981842)]

In [31]:
# Palabras que MENOS se relacionan con "Imperial":
w2v_model.wv.most_similar(negative=["imperial"], topn=5)

[('—una', 0.0493110828101635),
 ('buscar', 0.029083386063575745),
 ('fulham', 0.027314962819218636),
 ('preguntó', 0.026755427941679955),
 ('estrella', 0.022910205647349358)]

In [32]:
# Palabras que MENOS se relacionan con "Trántor":
w2v_model.wv.most_similar(negative=["trántor"], topn=5)

[('torpe', 0.048937730491161346),
 ('ansiedad', 0.04504131153225899),
 ('captó', 0.04076023027300835),
 ('leer', 0.038339197635650635),
 ('agente', 0.023380475118756294)]

### 5 - Visualizar agrupación de vectores

In [33]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [34]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

In [35]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab
fig.show() # esto para plotly en colab

In [36]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)